In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
TRAIN = 0
EVAL = 1
PRED = 2

In [2]:
X = tf.placeholder(tf.float32, shape=[None, 784])
reshaped_X = tf.reshape(X,[-1,28,28,1])
mode = tf.placeholder(tf.int32,shape = ())
Z = tf.placeholder(tf.float32, shape=[None, 16])
c = tf.placeholder(tf.float32, shape=[None, 10])

'''
D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 16])
c = tf.placeholder(tf.float32, shape=[None, 10])

G_W1 = tf.Variable(xavier_init([26, 256]))
G_b1 = tf.Variable(tf.zeros(shape=[256]))

G_W2 = tf.Variable(xavier_init([256, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


Q_W1 = tf.Variable(xavier_init([784, 128]))
Q_b1 = tf.Variable(tf.zeros(shape=[128]))

Q_W2 = tf.Variable(xavier_init([128, 10]))
Q_b2 = tf.Variable(tf.zeros(shape=[10]))

theta_Q = [Q_W1, Q_W2, Q_b1, Q_b2]
'''
xavier_init = tf.contrib.layers.xavier_initializer(uniform=False)

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def sample_c(m):
    return np.random.multinomial(1, 10*[0.1], size=m)

def shared_layers(input_layer):
    with tf.variable_scope("shared_conv",reuse = tf.AUTO_REUSE) as scope:
        conv1 = tf.layers.conv2d(
          inputs=input_layer,
          filters=32,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu,
          name = "shared_conv1",
          reuse = tf.AUTO_REUSE,
          kernel_initializer=xavier_init
        )
    
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
        conv2 = tf.layers.conv2d(
          inputs=pool1,
          filters=64,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu,
          name = "shared_conv2",
          reuse = tf.AUTO_REUSE,
          kernel_initializer=xavier_init
        )
    
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
        pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    return pool2_flat

def generator(z, c):
    #inputs = tf.concat(axis=1, values=[z, c])
    #G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    #G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    #G_prob = tf.nn.sigmoid(G_log_prob)
    with tf.variable_scope("generator") as scope:
        input_layer = tf.concat(axis=1,values = [z,c])
        dense_layer = tf.layers.dense(
            inputs = input_layer, 
            units = 7 * 7 * 64, 
            activation = tf.nn.relu,
            kernel_initializer=xavier_init
        )
        deconv1 = tf.reshape(dense_layer,[-1,7,7,64])
        print(deconv1.get_shape())
        norm_deconv1 = tf.contrib.layers.batch_norm(deconv1, is_training=mode==TRAIN)
        '''
        unpooling1 = tf.image.resize_images(
            images = norm_deconv1,
            size = tf.constant([14,14])
        )
        '''
        deconv2 = tf.layers.conv2d_transpose(
            inputs = norm_deconv1,
            filters = 32,
            kernel_size = [5,5],
            padding = "same",
            kernel_initializer=xavier_init,
            strides = 2
        )
        print(deconv2.get_shape())
        norm_deconv2 = tf.contrib.layers.batch_norm(deconv2, is_training=mode==TRAIN)
        '''
        unpooling2 = tf.image.resize_images(
            images = norm_deconv2,
            size = tf.constant([28,28])
        )
        '''
        deconv3 = tf.layers.conv2d_transpose(
            inputs = norm_deconv2,
            filters = 1,
            kernel_size = [2,2],
            padding = "same",
            kernel_initializer=xavier_init,
            strides = 2
        )
        print(deconv3.get_shape())
        norm_deconv3 = tf.contrib.layers.batch_norm(deconv3, is_training=mode==TRAIN)
        
    return norm_deconv3


def discriminator(x,D_mode):
    #D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    #D_logit = tf.matmul(D_h1, D_W2) + D_b2
    #D_prob = tf.nn.sigmoid(D_logit)
    pool2_flat = shared_layers(x)
    with tf.variable_scope("discriminator",reuse = tf.AUTO_REUSE) as scope:
        dense = tf.layers.dense(
            inputs=pool2_flat, 
            units=1024, 
            activation=tf.nn.relu,
            reuse = tf.AUTO_REUSE,
            name = "discriminator_dense1",
            kernel_initializer=xavier_init
        )
        dropout = tf.layers.dropout(
            inputs=dense, 
            rate=0.5, 
            training= D_mode == TRAIN
        )
        D_logit = tf.layers.dense(
            inputs=dropout,
            units=1,
            reuse = tf.AUTO_REUSE, 
            name = "discriminator_dense2",
            kernel_initializer=xavier_init
        )
        D_prob = tf.nn.sigmoid(D_logit)
    return D_prob


def Q(x,Q_mode):
    #Q_h1 = tf.nn.relu(tf.matmul(x, Q_W1) + Q_b1)
    #Q_prob = tf.nn.softmax(tf.matmul(Q_h1, Q_W2) + Q_b2)
    pool2_flat = shared_layers(x)
    with tf.variable_scope("Q") as scope:
        dense = tf.layers.dense(
            inputs=pool2_flat, 
            units=1024, 
            activation=tf.nn.relu,
            kernel_initializer=xavier_init
        )
        dropout = tf.layers.dropout(
            inputs=dense, 
            rate=0.5, 
            training= Q_mode == TRAIN
        )
        Q_logit = tf.layers.dense(inputs=dropout, units=10)
        Q_prob = tf.nn.softmax(Q_logit)
        
    return Q_prob


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [3]:
G_sample = generator(Z, c)
D_real = discriminator(reshaped_X,mode)
D_fake = discriminator(G_sample,mode)
Q_c_given_x = Q(G_sample,mode)

D_loss = -tf.reduce_mean(tf.log(D_real + 1e-8) + tf.log(1 - D_fake + 1e-8))
G_loss = -tf.reduce_mean(tf.log(D_fake + 1e-8))

cross_ent = tf.reduce_mean(-tf.reduce_sum(tf.log(Q_c_given_x + 1e-8) * c, 1))
ent = tf.reduce_mean(-tf.reduce_sum(tf.log(c + 1e-8) * c, 1))
Q_loss = cross_ent + ent

theta_shared = tf.trainable_variables(scope="shared_conv")
theta_D  = tf.trainable_variables(scope="discriminator") + theta_shared
theta_G  = tf.trainable_variables(scope="generator")
theta_Q  = tf.trainable_variables(scope="Q") + theta_shared

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
Q_solver = tf.train.AdamOptimizer().minimize(Q_loss, var_list=theta_G + theta_Q)

mb_size = 32
Z_dim = 16

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('2d_out/'):
    os.makedirs('2d_out/')

i = 0

for it in range(1000000):
    if it % 1000 == 0:
        Z_noise = sample_Z(16, Z_dim)

        idx = np.random.randint(0, 10)
        c_noise = np.zeros([16, 10])
        c_noise[range(16), idx] = 1

        samples = sess.run(G_sample,
                           feed_dict={Z: Z_noise, c: c_noise})

        fig = plot(samples)
        plt.savefig('2d_out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)
    Z_noise = sample_Z(mb_size, Z_dim)
    c_noise = sample_c(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss],
                              feed_dict={X: X_mb, Z: Z_noise, c: c_noise,mode:TRAIN})

    _, G_loss_curr = sess.run([G_solver, G_loss],
                              feed_dict={Z: Z_noise, c: c_noise})

    sess.run([Q_solver], feed_dict={Z: Z_noise, c: c_noise})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

(?, 7, 7, 64)
(?, 14, 14, 32)
(?, 28, 28, 1)
Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz
Iter: 0
D loss: 1.406
G_loss: 0.6279

Iter: 1000
D loss: 0.0128
G_loss: 9.765

Iter: 2000
D loss: 0.06027
G_loss: 6.847

Iter: 3000
D loss: 0.001576
G_loss: 7.143

Iter: 4000
D loss: 6.333e-08
G_loss: 18.42

Iter: 5000
D loss: -0.0
G_loss: 18.42

Iter: 6000
D loss: -0.0
G_loss: 18.42

Iter: 7000
D loss: -0.0
G_loss: 18.42

Iter: 8000
D loss: -0.0
G_loss: 18.42



KeyboardInterrupt: 

[]